In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/driving_dataset.zip", 'r')
zip_ref.extractall("/content/data")
zip_ref.close()

In [ ]:
import pandas
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
import keras

data = pandas.read_csv('/content/drive/My Drive/data.txt',header = None,sep=' ' )

In [ ]:
data = data.values

In [ ]:
x_data = data[:,0]
y_data = np.array([float(i) for i in data[:,-1]])
print(x_data.shape,y_data.shape)

(45406,) (45406,)


In [ ]:
l = (int)(x_data.shape[0]*0.8)
xtrain = x_data[:l]
ytrain = y_data[:l]
xtest = x_data[l:]
ytest = y_data[l:]
print(xtrain.shape,xtest.shape,ytrain.shape,ytest.shape)

(36324,) (9082,) (36324,) (9082,)


In [ ]:

import random
c = list(zip(xtrain,ytrain))
random.shuffle(c)
xtrain,ytrain = zip(*c)


c = list(zip(xtest,ytest))
random.shuffle(c)
xtest,ytest = zip(*c)

In [ ]:
from keras.preprocessing import image

def LoadBatch(batch_size,i,xdata,ydata):
  xd = []
  yd = []
  Pointer = 0;
  if i==len(xdata)//batch_size - 1:
      batch_size = len(xdata)%batch_size
  while Pointer<batch_size:
    img = image.load_img("/content/data/driving_dataset/"+xdata[(Pointer+i*batch_size)%l],color_mode='rgb',target_size=[200,200])
    img = image.img_to_array(img)/255.0
    xd.append(img)
    yd.append(ydata[(Pointer+i*batch_size)%l]*np.pi/180)
    Pointer = Pointer+1
  return np.array(xd),np.array(yd)

In [ ]:
from keras.layers import Conv2D,Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, MaxPool2D
from keras.optimizers import Adam
from keras.models import Sequential
from keras import regularizers
from keras.optimizers import Adam

# from keras.adv

In [ ]:
model = Sequential()

model.add(Conv2D(24,(5,5),activation='relu',input_shape=(200,200,3),kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(36,(5,5),activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(48,(3,3),activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64,(3,3),activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(128,(3,3),activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.3))

model.add(Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))

model.add(Dense(1,activation='tanh',kernel_regularizer=regularizers.l2(0.01)))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 196, 196, 24)      1824      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 98, 98, 24)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 98, 98, 24)        96        
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 94, 94, 36)        21636     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 47, 47, 36)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 47, 47, 36)        144       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 45, 45, 48)       

In [ ]:
model.compile(optimizer = Adam(0.0001),loss = 'mse',metrics=['mae'])

In [ ]:
epochs = 5
batch_size = 128
num = len(xtrain)

c=0
l = len(xtest)

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(num//batch_size)):
        xs, ys = LoadBatch(batch_size,i,xtest,ytest)
        ys.reshape(-1,1)

        model.fit(xs,ys,verbose=0)

        if i%10 == 0:
            score_train = model.evaluate(xs,ys,verbose=0)
            xs1, ys1 = LoadBatch(batch_size,c,xtrain,ytrain)
            c+=1
            ys1.reshape(-1,1)
            score_test = model.evaluate(xs1,ys1,verbose=0)
            print("Epoch:%d Step:%d Train loss:%f Train mae:%f Test loss:%f Test mae:%f\n"%(epoch+1,i,score_train[0],score_train[1],score_test[0],score_test[1]))

        if i%100 == 0:
            model.save_weights("/content/drive/My Drive/gkl7.h5")


Epoch:1 Step:0 Train loss:5.731159 Train mae:0.293378 Test loss:6.185930 Test mae:0.328742

Epoch:1 Step:10 Train loss:5.642132 Train mae:0.265078 Test loss:5.806211 Test mae:0.351447

Epoch:1 Step:20 Train loss:5.648902 Train mae:0.295390 Test loss:6.124649 Test mae:0.416094

Epoch:1 Step:30 Train loss:5.493180 Train mae:0.260992 Test loss:5.375972 Test mae:0.263368

Epoch:1 Step:40 Train loss:5.242342 Train mae:0.168602 Test loss:5.491564 Test mae:0.336270

Epoch:1 Step:50 Train loss:5.193392 Train mae:0.267289 Test loss:5.228124 Test mae:0.267044

Epoch:1 Step:60 Train loss:5.240770 Train mae:0.292058 Test loss:5.317111 Test mae:0.342019

Epoch:1 Step:70 Train loss:5.137605 Train mae:0.424827 Test loss:5.214151 Test mae:0.449278

Epoch:1 Step:80 Train loss:4.981936 Train mae:0.354875 Test loss:5.101775 Test mae:0.402843

Epoch:1 Step:90 Train loss:5.008132 Train mae:0.453213 Test loss:5.258098 Test mae:0.509665

Epoch:1 Step:100 Train loss:4.819540 Train mae:0.354652 Test loss:5.063